In [1]:
def make_features_dic(fname):
    with open(fname) as file:
        return {line.rstrip('\n'): i for i, line in enumerate(file)}

In [2]:
def make_vector_x(title, features_dic):
    import re
    from nltk.corpus import stopwords
    import snowballstemmer
    
    vector_x = np.zeros([1, len(features_dic)])
    
    # 文字種の統一
    title = title.lower()
            
    # 数字の置き換え -> 除去
    title = re.sub(r'[0-9]+', '', title)
    
    # '-'を' 'に変換
    title = title.replace('-', ' ')
    
    words = title.split()
    
    # ストップワードの除去
    stop_words = stopwords.words('english')
    words2 = [word for word in words if word not in stop_words]
    words = words2       
    
    # ステミング処理
    stemmer = snowballstemmer.stemmer('english')
    words2 = [stemmer.stemWord(word) for word in words]
    words = words2
    
    # 記号の除去
    words2 = [word for word in words if word.islower()]
    words = words2
    
    for word in words:
        if word in features_dic.keys():
            vector_x[0, features_dic[word]] = 1
            
    return vector_x

In [3]:
def softmax(queue): # ソフトマックス関数
    return np.exp(queue) / np.sum(np.exp(queue), axis = 1)[:, np.newaxis]

In [4]:
def predeict_category(vector_x, fname):
    w_and_b = np.load(fname)
    queue_w = w_and_b[:-1]
    vector_b = w_and_b[-1]
    phi = softmax(np.dot(vector_x, queue_w) + vector_b)
    class_label = np.argmax(phi, axis = 1)
    
    if class_label == 0:
        category = 'b'
    elif class_label == 1:
        category = 't'
    elif class_label == 2:
        category = 'e'
    elif class_label == 3:
        category = 'm'
        
    return category, phi[:,class_label]

In [5]:
import numpy as np

title = input('enter title: ')
features_dic = make_features_dic('train.feature.txt')
vector_x = make_vector_x(title, features_dic)
pred_category, probability = predeict_category(vector_x, 'results/52_result.npy')

print('カテゴリ: {0}\t予測確率: {1}'.format(pred_category, probability))

enter title: Disney Announce Plans For Two New Sequels: 'The Incredibles 2' and 'Cars 3'
カテゴリ: e	予測確率: [[0.9982954]]
